In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as f
import re
import time

In [2]:
df = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5'),
    ('id_cliente-3',  'cat-6, cat-7'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9'),
    ('id_cliente-9',  'cat-1'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

In [3]:
df2 = spark.createDataFrame([
    ('id_cliente-1',  'cat-1, cat-2, cat-3, cat-15'),
    ('id_cliente-2',  'cat-1, cat-4, cat-5, cat-11, cat-14'),
    ('id_cliente-3',  'cat-4, cat-14, cat-15'),
    ('id_cliente-4',  'cat-1, cat-2, cat-7, cat-10'),
    ('id_cliente-5',  'cat-8, cat-10, cat-11'),
    ('id_cliente-6',  'cat-1, cat-9, cat-10, cat-11, cat-13'),
    ('id_cliente-7',  'cat-1, cat-4, cat-5, cat-10'),
    ('id_cliente-8',  'cat-7, cat-9, cat-12, cat-13, cat-14'),
    ('id_cliente-9',  'cat-2'),
    ('id_cliente-10', 'cat-1, cat-2, cat-3, cat-4, cat-5, cat-6, cat-7, cat-8, cat-10')
], ['id_cliente', 'categorias'])

In [4]:
def pegaNumero(x):
  return(int(re.findall(r'\d+', x)[0]))

def transforma(x):
  return('client-uuid-'+str(x))

pegaNumeroUDF = udf(pegaNumero, IntegerType())
transformaUDF = udf(transforma, StringType())

In [5]:
start = time.time()
x1 = df.select('id_cliente', explode(split("categorias",",")).alias("opa")).withColumn("opa", trim(col("opa")))
x1 = x1.withColumn('id', pegaNumeroUDF("id_cliente"))
x1 = x1.withColumn('opa_id', pegaNumeroUDF("opa"))
x1 = x1.groupBy('id_cliente','id').pivot('opa_id').count().na.fill(0).sort('id', ascending=True)
x1 = x1.withColumn('id', transformaUDF("id"))
replacements = {c:'cat-'+str(c) for c in x1.columns if c.isnumeric()}
x1 = x1.select([col(c).alias(replacements.get(c, c)) for c in x1.columns])
x1.drop('id_cliente').show()
print("DF1 - " , time.time() - start)

start = time.time()
x1 = df2.select('id_cliente', explode(split("categorias",",")).alias("opa")).withColumn("opa", trim(col("opa")))
x1 = x1.withColumn('id', pegaNumeroUDF("id_cliente"))
x1 = x1.withColumn('opa_id', pegaNumeroUDF("opa"))
x1 = x1.groupBy('id_cliente','id').pivot('opa_id').count().na.fill(0).sort('id', ascending=True)
x1 = x1.withColumn('id', transformaUDF("id"))
replacements = {c:'cat-'+str(c) for c in x1.columns if c.isnumeric()}
x1 = x1.select([col(c).alias(replacements.get(c, c)) for c in x1.columns])
x1.drop('id_cliente').show()
print("DF2 - " , time.time() - start)

+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
 id|cat-1|cat-2|cat-3|cat-4|cat-5|cat-6|cat-7|cat-8|cat-9|cat-10|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+
 client-uuid-1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0|
 client-uuid-2| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0|
 client-uuid-3| 0| 0| 0| 0| 0| 1| 1| 0| 0| 0|
 client-uuid-4| 1| 1| 0| 0| 0| 0| 1| 0| 0| 1|
 client-uuid-5| 0| 0| 0| 0| 0| 0| 0| 1| 0| 1|
 client-uuid-6| 1| 0| 0| 0| 0| 0| 0| 0| 1| 1|
 client-uuid-7| 1| 0| 0| 1| 1| 0| 0| 0| 0| 1|
 client-uuid-8| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0|
 client-uuid-9| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
client-uuid-10| 1| 1| 1| 1| 1| 1| 1| 1| 0| 1|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+

DF1 - 4.652985572814941
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
 id|cat-1|cat-2|cat-3|cat-4|cat-5|cat-6|cat-7|cat-8|cat-9|cat-10|cat-11|cat-12|cat-13|cat-14|cat-15|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+
 client-uuid-1| 1| 1| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1|
 client-uuid-2| 1| 0| 0| 1| 1| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0|
 client-uuid-3| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 1|
 client-uuid-4| 1| 1| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0|
 client-uuid-5| 0| 0| 0| 0| 0| 0| 0| 1| 0| 1| 1| 0| 0| 0| 0|
 client-uuid-6| 1| 0| 0| 0| 0| 0| 0| 0| 1| 1| 1| 0| 1| 0| 0|
 client-uuid-7| 1| 0| 0| 1| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0|
 client-uuid-8| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0| 0| 1| 1| 1| 0|
 client-uuid-9| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
client-uuid-10| 1| 1| 1| 1| 1| 1| 1| 1| 0| 1| 0| 0| 0| 0| 0|
+--------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+

DF2 - 4.411900997161865